In [16]:
import numpy as np

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


In [9]:
def train_breast_cancer(config):
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)

    train_x, test_x, train_y, test_y = train_test_split(
        data, labels, test_size=0.25
    )

    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)

    results = {}

    bst = xgb.train(
        params=config,
        dtrain=train_set,
        evals=[(test_set, 'evaluation1')],
        evals_result = results,
        verbose_eval=False,
    )

    return results

In [18]:
config = {
    'objective': 'binary:logistic',
    'eval_metric':['logloss', 'error']
}

results = train_breast_cancer(config=config)
accuracy = 1.0-np.mean(results['evaluation1']['error'])
logloss = np.mean(results['evaluation1']['logloss'])
print(f'Accuracy: {accuracy:.4f} \nLog Loss: {logloss:.4f}')

Accuracy: 0.9727 
Log Loss: 0.1844


In [ ]:
config = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "max_depth": 2,
    "min_child_weight": 0,
    "subsample": 0.8,
    "eta": 0.2,
}

results = train_breast_cancer(config=config)
accuracy = 1.0-np.mean(results['evaluation1']['error'])
logloss = np.mean(results['evaluation1']['logloss'])
print(f'Accuracy: {accuracy:.4f} \nLog Loss: {logloss:.4f}')

Accuracy: 0.8769 
Log Loss: 0.3345


### Tuning the config parameters

In [34]:
from ray import train, tune

def train_breast_cancer(config):
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)

    train_x, test_x, train_y, test_y = train_test_split(
        data, labels, test_size=0.25
    )

    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)

    results = {}

    bst = xgb.train(
        params=config,
        dtrain=train_set,
        evals=[(test_set, 'eval')],
        evals_result = results,
        verbose_eval=False,
    )

    accuracy = 1.0 - results["eval"]["error"][-1]
    train.report({"mean_accuracy": accuracy, "done": True})


config = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "max_depth": tune.randint(1, 9),
    "min_child_weight": tune.choice([1, 2, 3]),
    "subsample": tune.uniform(0.5, 1.0),
    "eta": tune.loguniform(1e-4, 1e-1),
}
tuner = tune.Tuner(
    train_breast_cancer,
    tune_config=tune.TuneConfig(metric='mean_accuracy', mode='max',num_samples=10),
    param_space=config,
)
results = tuner.fit()

2024-11-12 17:06:37,133	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/JCA/ray_results/train_breast_cancer_2024-11-12_17-06-23' in 0.0605s.
2024-11-12 17:06:37,177	INFO tune.py:1041 -- Total run time: 13.95 seconds (13.83 seconds for the tuning loop).


In [43]:
results.get_best_result().metrics_dataframe.T

,0
mean_accuracy,0.965035
done,True
timestamp,1731402396
checkpoint_dir_name,None
training_iteration,1
trial_id,63aa5_00009
date,2024-11-12_17-06-36
time_this_iter_s,0.578726
time_total_s,0.578726
pid,15556


### Early Stopping

In [44]:
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback

In [47]:
def train_breast_cancer(config: dict):
    data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.25)

    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    test_set = xgb.DMatrix(test_x, label=test_y)

    # Train the classifier, using the Tune callback
    xgb.train(
        config,
        train_set,
        evals=[(test_set, "eval")],
        verbose_eval=False,
        # `TuneReportCheckpointCallback` defines the checkpointing frequency and format.
        callbacks=[TuneReportCheckpointCallback(frequency=1)],
    )

def get_best_model_checkpoint(results):
    best_result = results.get_best_result()

    # `TuneReportCheckpointCallback` provides a helper method to retrieve the
    # model from a checkpoint.
    best_bst = TuneReportCheckpointCallback.get_model(best_result.checkpoint)

    accuracy = 1.0 - best_result.metrics["eval-error"]
    print(f"Best model parameters: {best_result.config}")
    print(f"Best model total accuracy: {accuracy:.4f}")
    return best_bst

def tune_xgboost(smoke_test=False):
    search_space = {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "max_depth": tune.randint(1, 9),
        "min_child_weight": tune.choice([1, 2, 3]),
        "subsample": tune.uniform(0.5, 1.0),
        "eta": tune.loguniform(1e-4, 1e-1),
    }

    # This will enable aggressive early stopping of bad trials.
    scheduler = ASHAScheduler(
        max_t = 10, grace_period=1, reduction_factor=2 # 10 training iterations
    )

    tuner = tune.Tuner(
        train_breast_cancer,
        tune_config=tune.TuneConfig(
            metric="eval-logloss",
            mode="min",
            scheduler=scheduler,
            num_samples=1 if smoke_test else 10,
        ),
        param_space=search_space,
    )
    results = tuner.fit()
    return results


results = tune_xgboost(smoke_test=False)

2024-11-12 17:32:48,690	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/JCA/ray_results/train_breast_cancer_2024-11-12_17-32-34' in 0.0981s.
2024-11-12 17:32:48,752	INFO tune.py:1041 -- Total run time: 14.56 seconds (14.38 seconds for the tuning loop).


In [49]:
# Load the best model checkpoint.
best_bst = get_best_model_checkpoint(results)

Best model parameters: {'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.6301194488488465, 'eta': 0.038338617965135324}
Best model total accuracy: 0.9161


In [50]:
# best_bst.predict(...)

### Using fractional GPUs

In [ ]:
def tune_xgboost_rsc(smoke_test=False):
    search_space = {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        "max_depth": tune.randint(1, 9),
        "min_child_weight": tune.choice([1, 2, 3]),
        "subsample": tune.uniform(0.5, 1.0),
        "eta": tune.loguniform(1e-4, 1e-1),
    }

    # This will enable aggressive early stopping of bad trials.
    scheduler = ASHAScheduler(
        max_t = 10, grace_period=1, reduction_factor=2 # 10 training iterations
    )

    tuner = tune.Tuner(

        # If you run 20 trials, each task will use 5% of the available GPU memory
        tune.with_resources(train_breast_cancer,
                            resources={'cpu':1, 'gpu':0.05}),
        tune_config=tune.TuneConfig(
            metric="eval-logloss",
            mode="min",
            scheduler=scheduler,
            num_samples=1 if smoke_test else 20,
        ),
        param_space=search_space,
    )
    results = tuner.fit()
    return results


results_gpu_20 = tune_xgboost_rsc(smoke_test=False)

2024-11-12 17:38:36,125	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/JCA/ray_results/train_breast_cancer_2024-11-12_17-38-06' in 0.0784s.
2024-11-12 17:38:36,166	INFO tune.py:1041 -- Total run time: 29.45 seconds (29.30 seconds for the tuning loop).


In [53]:
# Load the best model checkpoint.
best_bst = get_best_model_checkpoint(results_gpu_20)

Best model parameters: {'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.9545523772371498, 'eta': 0.08528134780427145}
Best model total accuracy: 0.9510
